In [1]:
from warnings import filterwarnings
filterwarnings(action='ignore',
                        category=DeprecationWarning,
                        module='stable_baselines')
import gym
import PortfolioAllocationGym
import numpy as np
from stable_baselines import A2C
from stable_baselines.common.policies import MlpLnLstmPolicy, MlpPolicy, MlpLstmPolicy
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.env_checker import check_env
from stable_baselines.bench import Monitor
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

C:\Users\kbine\anaconda3\envs\stable2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kbine\anaconda3\envs\stable2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kbine\anaconda3\envs\stable2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kbine\anaconda3\envs\stable2\lib\site-packages\tensorflow\

In [2]:
env_kwargs = {'filename':'sp500.csv',
    'date_from':'2008-01-01',
    'date_to':'2017-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':500,
    'random_sample':False,
    'reward_function':'sharpe'}

train_env = gym.make('PortfolioAllocation-v0', **env_kwargs)
train_env = Monitor(train_env, 'monitor')

In [3]:
check_env(train_env)

C:\Users\kbine\anaconda3\envs\stable2\lib\site-packages\stable_baselines\common\env_checker.py:210: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn("We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [7]:
venv, obs = train_env.get_sb_env()


In [8]:
'''
model_kwargs =   {'gamma': 0.9999,
    'normalize_advantage': False,
    'max_grad_norm': 0.7,
    'use_rms_prop': False,
    'gae_lambda': 0.92,
    'n_steps': 10,
    'learning_rate': 0.0038610316815332825,
    'ent_coef': 0.012292116134058367,
    'vf_coef': 0.7960524189522955,
    'policy_kwargs': dict(
        log_std_init=-3.353286611055509,
        ortho_init= False,
        activation_fn=torch.nn.modules.activation.ReLU,
        net_arch=[dict(pi=[64, 64, 64], vf=[64, 64, 64])])
    }
'''

a2c_model = A2C(policy = MlpLstmPolicy, tensorboard_log="tensorboard",
                env = venv)
                #**model_kwargs)


In [5]:
from PortfolioAllocationGym.callbacks import TensorBoardCallback as tbc
from datetime import datetime

In [9]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 reward: 0.457                 sharpe: 0.457                  cum. rtns: 268.078                 portf val: 3,680,780.35
day: 2516                 reward: 0.464                 sharpe: 0.464                  cum. rtns: 273.105                 portf val: 3,731,053.64
day: 2516                 reward: 0.464                 sharpe: 0.464                  cum. rtns: 269.839                 portf val: 3,698,387.15
day: 2516                 reward: 0.464                 sharpe: 0.464                  cum. rtns: 268.652                 portf val: 3,686,520.40
day: 2516                 reward: 0.465                 sharpe: 0.465                  cum. rtns: 268.012                 portf val: 3,680,120.10
mean_reward:24.54 +/- 8.68


In [12]:
total_timesteps = 2000 * (len(venv.venv.envs[0].data.date.unique())-1)
#total_timesteps = 500 * (len(venv.data.date.unique())-1)
trained_a2c_model= a2c_model.learn(total_timesteps=total_timesteps, #callback=tbc.TensorBoardCallback(),
                                   tb_log_name='A2C'+datetime.now().strftime("%H-%M"))


day: 2516                 reward: 0.384                 sharpe: 0.384                  cum. rtns: 282.221                 portf val: 3,822,208.16
day: 2516                 reward: 0.390                 sharpe: 0.390                  cum. rtns: 276.027                 portf val: 3,760,268.93
day: 2516                 reward: 0.354                 sharpe: 0.354                  cum. rtns: 282.357                 portf val: 3,823,574.65
day: 2516                 reward: 0.377                 sharpe: 0.377                  cum. rtns: 265.349                 portf val: 3,653,487.32
day: 2516                 reward: 0.331                 sharpe: 0.331                  cum. rtns: 245.715                 portf val: 3,457,147.97
day: 2516                 reward: 0.369                 sharpe: 0.369                  cum. rtns: 236.717                 portf val: 3,367,165.06
day: 2516                 reward: 0.434                 sharpe: 0.434                  cum. rtns: 269.597                 p

In [15]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
mean_reward:nan +/- nan


In [ ]:
'''

eval_kwargs = {'filename':'sp500.csv',
    'date_from':'2018-01-01',
    'date_to':'2020-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5,
    'reward_function':'daily_returns'}

eval_env =  Monitor(gym.make('PortfolioAllocation-v0', **eval_kwargs))

mean_reward, std_reward = evaluate_policy(trained_a2c_model, eval_env, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
'''

In [14]:
trained_a2c_model.save('ltsm_500_2000_collapse')

In [25]:
dir(trained_a2c_model.env.venv.envs[0].info)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [27]:
trained_a2c_model.env.venv.envs[0].info['actions_history']


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])